**Proyecto:** INE Incidentes de Transito.

**Objetivo:** Consultas con los principales resultados.

**Dependencias / Librerias:** notebook transform

**Ambiente de Ejecución:** Databricks, Desarrollo

---
| Fecha | Desarrollador | Descripcion |
| :--- | :--- | :--- |
| 10/NOV/2025 | Ruben Coloma | Emision Inicial |

In [0]:
from pyspark.sql import functions as F

**VICTIMAS DE HECHOS DE TRANSITO POR DEPARTAMENTO (EXCLUYENDO DEPARTAMENTO DE GUATEMALA) _POR_ MES DURANTE 2023**

In [0]:
# Cargar las tablas del catálogo
hechos = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados")
dim_depto = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_depto_ocu")
dim_mes = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mes_ocu")

# Unir las tablas
df = (
    hechos
    .join(dim_depto, "id_depto_ocu", "left")
    .join(dim_mes, "id_mes_ocurrio", "left")
)

# Excluir el departamento de "Guatemala"
df = df.filter(F.col("depto_ocu") != "Guatemala")

# Agrupar por departamento y mes, contando incidentes
df_agregado = (
    df.groupBy("depto_ocu", "mes_mezcla")
      .agg(F.count("*").alias("total_incidentes"))
)

# Pivotear: columnas = meses, filas = departamentos
df_pivot = (
    df_agregado.groupBy("depto_ocu")
               .pivot("mes_mezcla")
               .agg(F.sum("total_incidentes"))
)

# Agregar columna TOTAL (suma de todos los meses)
# Se suman dinámicamente todas las columnas excepto depto_ocu
cols_mes = [c for c in df_pivot.columns if c not in ["depto_ocu"]]
df_final = df_pivot.withColumn("TOTAL", sum(F.coalesce(F.col(c), F.lit(0)) for c in cols_mes))

# Renombrar columna depto_ocu a MES (como pediste)
df_final = df_final.withColumnRenamed("depto_ocu", "Departamento")

# Mostrar el resultado
display(df_final.orderBy("TOTAL"))

**VICTIMAS DE HECHOS DE TRANSITO POR MUNICIPIO DEL DEPARTAMENTO DE GUATEMALA EXCLUYENDO EL MUNICIPIO DE GUATEMALA POR MES DURANTE 2023**

In [0]:
# Cargar las tablas del catálogo
dim_municipio = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mupio_ocu")


# Unir las tablas
df = (
    hechos
    .join(dim_depto, "id_depto_ocu", "left")
    .join(dim_municipio, "id_mupio_ocu", "left")
    .join(dim_mes, "id_mes_ocurrio", "left")
)

# Solo incluir el departamento de "Guatemala"
df = df.filter(F.col("depto_ocu") == "Guatemala")

# Excluir el municipio de "Guatemala"
df = df.filter(F.col("mupio_ocu") != "Guatemala")

# Agrupar por departamento y mes, contando incidentes
df_agregado = (
    df.groupBy("mupio_ocu", "mes_mezcla")
      .agg(F.count("*").alias("total_incidentes"))
)

# Pivotear: columnas = meses, filas = departamentos
df_pivot = (
    df_agregado.groupBy("mupio_ocu")
               .pivot("mes_mezcla")
               .agg(F.sum("total_incidentes"))
)

# Agregar columna TOTAL (suma de todos los meses)
# Se suman dinámicamente todas las columnas excepto depto_ocu
cols_mes = [c for c in df_pivot.columns if c not in ["mupio_ocu"]]
df_final = df_pivot.withColumn("TOTAL", sum(F.coalesce(F.col(c), F.lit(0)) for c in cols_mes))

# Renombrar columna depto_ocu a MES (como pediste)
df_final = df_final.withColumnRenamed("mupio_ocu", "Municipio")

# Guardar los datos en Gold
#df_final.write.format("delta").mode("overwrite").saveAsTable("cat_ine_trafico_dev.gold_ine_trafico_dev.resumen_victimas_depto_guate")


# Mostrar el resultado
display(df_final.orderBy("Municipio"))

**VICTIMAS DE HECHOS DE TRANSITO POR ZONA DEL MUNICIPIO DE GUATEMALA POR MES DURANTE 2023**

In [0]:
# Cargar tabla de hechos
hechos = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados")

# Unir las tablas
df = (
    hechos
    .join(dim_depto, "id_depto_ocu", "left")
    .join(dim_municipio, "id_mupio_ocu", "left")
    .join(dim_mes, "id_mes_ocurrio", "left")
)

# Solo incluir el departamento de "Guatemala"
df = df.filter(F.col("depto_ocu") == "Guatemala")

# Solo incluir el municipio de "Guatemala"
df = df.filter(F.col("mupio_ocu") == "Guatemala")

# Reemplazar zona_ocu = -1 por "Ignorada"
df = df.withColumn(
    "zona_ocu",
    F.when(F.col("zona_ocu") == -1, F.lit("Ignorada"))
     .otherwise(F.col("zona_ocu").cast("string"))  # convertimos a string para uniformidad
)

# Agrupar por departamento y mes, contando incidentes
df_agregado = (
    df.groupBy("zona_ocu", "mes_mezcla")
      .agg(F.count("*").alias("total_incidentes"))
)

# Pivotear: columnas = meses, filas = departamentos
df_pivot = (
    df_agregado.groupBy("zona_ocu")
               .pivot("mes_mezcla")
               .agg(F.sum("total_incidentes"))
)

# Agregar columna TOTAL (suma de todos los meses)
# Se suman dinámicamente todas las columnas excepto depto_ocu
cols_mes = [c for c in df_pivot.columns if c not in ["zona_ocu"]]
df_final = df_pivot.withColumn("TOTAL", sum(F.coalesce(F.col(c), F.lit(0)) for c in cols_mes))

# Renombrar columna depto_ocu a MES (como pediste)
df_final = df_final.withColumnRenamed("zona_ocu", "Zona")

# Guardar los datos en Gold
#df_final.write.format("delta").mode("overwrite").saveAsTable("cat_ine_trafico_dev.gold_ine_trafico_dev.resumen_victimas_muni_guate")

# Mostrar el resultado
display(df_final.orderBy("Zona"))

**FALLECIDOS Y LECIONADOS EN HECHOS DE TRANSITO POR GENERO DURANTE 2023**

In [0]:

# Cargar tablas del catálogo
hechos = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados")
dim_fall_les = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_fall_les")
dim_sexo = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_sexo_per")
dim_mes = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mes_ocu")

# Unir las tablas
df = (
    hechos
    .join(dim_fall_les, "id_fall_les", "left")
    .join(dim_sexo, "id_sexo_per", "left")
    .join(dim_mes, "id_mes_ocurrio", "left")
)

# Generar columnas de conteo condicional
#    Fallecidos = registros donde fall_les = 'FALLECIDO'
#    Lesionados = registros donde fall_les = 'LESIONADO'
df_agregado = (
    df.groupBy("mes_mezcla", "sexo_per")
      .agg(
          F.sum(F.when(F.col("fall_les") == "Fallecido", 1).otherwise(0)).alias("Fallecidos"),
          F.sum(F.when(F.col("fall_les") == "Lesionado", 1).otherwise(0)).alias("Lesionados")
      )
)

df_renombrado = df_agregado \
    .withColumnRenamed("mes_mezcla", "Mes") \
    .withColumnRenamed("sexo_per", "Sexo") \
    .withColumnRenamed("Fallecidos", "Total Fallecidos") \
    .withColumnRenamed("Lesionados", "Total Lesionados")

display(df_renombrado)

**FALLECIDOS Y LECIONADOS POR VEHICULO EN HECHOS DE TRANSITO POR GENERO DURANTE 2023**

In [0]:
# COLUMNAS: MES, GENERO, VEHICULO, FALLECIDOS, LECIONADOS

**FALLECIDOS Y LECIONADOS POR TIPO DE EVENTO EN HECHOS DE TRANSITO POR GENERO DURANTE 2023**

In [0]:
# COLUMNAS: MES, GENERO, TIPO EVENTO, FALLECIDOS, LECIONADOS